## Hand token relabelling script

Because of (a) the non-standard nature of token corrections and (b) the relatively few number of them avaliable, we used a hand re-annotation process to identify the changes needing to be made to each line of interest. This Notebook provides a basic user interface to carry out that process, and saves the outputs to a csv file which then can be used to re-label the corpus. 

In [ ]:
import fileinput
import sys
import glob 
import pandas as pd
import numpy as np

In [ ]:

filenames = {"test":  '../original_corpus/eng.testb',
             "dev" :  '../original_corpus/eng.testa',
             "train": '../original_corpus/eng.train'}
folds = ['test','dev','train']
dtypes_in = {'correct_line':str}

log = []
log.append("line, num_added")

files = {}
for fold,filename in filenames.items(): 
    print(fold,filename)
    with open(filename, 'r') as file:
        files[fold] = file.readlines()

filenames
#read dataframe from file
corrections = pd.read_csv("../corrected_labels/token_edits.csv",dtype = dtypes_in)
corrections = corrections[['fold','line_no','doc_offset','corpus_span','correct_span','correct_line']]
corrections.head()


In [ ]:
 
def print_with_numbering(file, s, e):
        for i in range(s,e):
            print(i, file[i])
        
def show_lines(line, file, line_end = None):
    line_end = line +2 if (line_end== None) else line_end
    print_with_numbering(file,line-1,line_end+1 )

def get_corpus_line(line):
    return data[line]
    


In [ ]:
# run this to reset count
curr_row = 0; 
fold = "dev"

In [ ]:
# for now just display
# when this bool is set to true, replace line with given text
prompt_for_correct = True;

for _,row in (corrections.loc[curr_row:curr_row+2]).iterrows():
    print(row.to_list())
    show_lines(row.line_no, files[row.fold])
        
if(prompt_for_correct):
    print("enter correct input for first entry")
    inp = input()
    if(inp == 'delete_entry'):
        corrections.drop(index=curr_row,axis = 1);
    else: 
        inp = inp.replace("\\n","\n")
        corrections.at[curr_row,'correct_line'] = inp
        print("\n line gotten: \n")
        print(inp)

curr_row += 1 

In [ ]:
# curr_row -=1
display(curr_row)
corrections.head(curr_row).tail()

In [ ]:
# sometimes two errors in the labels were created by a single token error - one for the object in front of the error and one for the object after it. 
# we want to remove these duplicates for the final processing. 
# also remove non-filled out lines. 
corrections_preened = corrections.drop_duplicates(["fold","line_no", "doc_offset"], ignore_index=True); 
corrections_preened = corrections_preened[corrections_preened["correct_line"] != '']
corrections_preened.set_index("line_no", inplace = True)

In [ ]:
with open("../corrected_labels/token_corrections.json",'w') as file:
    corrections_preened.to_json(file)
print("printed to file")